# Imports

In [1]:
# 导入需要的库    Импорт требуемых библиотек
import keras
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import random
import re
import string
from torchsummary import summary

In [2]:
# 调用 GPU 加速    Вызов GPU-ускорения
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except:
        pass

# Dataset

In [3]:
# 文件路径    Путь к файлу
file_path = r"C:\Users\lcf14\Desktop\homework\Machine_Learning_appli\有俄语-英语对应句 - 2024-10-31.tsv"

In [4]:
# 读取文件并设置列名，忽略有问题的行    Чтение файла и установка имен столбцов, пропуская строки с ошибками
data = pd.read_csv(file_path, sep="\t", header=None, names=["id_rus", "text_rus", "id_eng", "text_eng"], on_bad_lines='skip')
#data = data[:100]  # 仅选择前5000行数据

# 输出前几行查看数据    Вывод первых нескольких строк для просмотра данных
print(data[['id_rus', 'text_rus', 'id_eng', 'text_eng']].head())

   id_rus                                           text_rus  id_eng  \
0     243  Один раз в жизни я делаю хорошее дело... И оно...    3257   
1    5409                      Давайте что-нибудь попробуем!    1276   
2    5410                               Мне пора идти спать.    1277   
3    5411                                    Что ты делаешь?   16492   
4    5411                                    Что ты делаешь?  511884   

                                            text_eng  
0  For once in my life I'm doing a good deed... A...  
1                               Let's try something.  
2                             I have to go to sleep.  
3                                What are you doing?  
4                                  What do you make?  


In [5]:
# 定义一个函数，用于去除文本中的标点符号    Определение функции для удаления пунктуации из текста
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))  # 使用translate方法删除标点符号    Использование метода translate для удаления пунктуации

# 对俄语文本列应用去除标点符号的函数    Применение функции удаления пунктуации к столбцу с русским текстом
data['text_rus'] = data['text_rus'].apply(remove_punctuation)

# 对英语文本列应用去除标点符号的函数    Применение функции удаления пунктуации к столбцу с английским текстом
data['text_eng'] = data['text_eng'].apply(remove_punctuation)

# 打印数据框中俄语和英语文本的前几行    Вывод первых строк текста на русском и английском языках
print(data[['id_rus', 'text_rus', 'id_eng', 'text_eng']].head())


   id_rus                                           text_rus  id_eng  \
0     243  Один раз в жизни я делаю хорошее дело И оно бе...    3257   
1    5409                        Давайте чтонибудь попробуем    1276   
2    5410                                Мне пора идти спать    1277   
3    5411                                     Что ты делаешь   16492   
4    5411                                     Что ты делаешь  511884   

                                            text_eng  
0  For once in my life Im doing a good deed And i...  
1                                 Lets try something  
2                              I have to go to sleep  
3                                 What are you doing  
4                                   What do you make  


In [6]:
# 初始化一个GPT2的分词器    Инициализация токенизатора GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 添加特殊标记，包括起始、结束和填充标记    Добавление специальных токенов: начало, конец и заполнение
tokenizer.add_special_tokens({
    "bos_token": "<BOS>",  # 起始标记    Токен начала предложения
    "eos_token": "<EOS>",  # 结束标记    Токен конца предложения
    "pad_token": "<PAD>"   # 填充标记    Токен заполнения
})

# 将填充标记的ID设为结束标记的ID    Установка идентификатора токена заполнения таким же, как у токена конца предложения
tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
# 定义数据集生成函数    Определение функции для генерации датасета
def Sentences_Dataset(sentence_pairs, tokenizer, max_len=32):
    inputs = []  # 输入张量列表    Список тензоров для входов
    outputs = []  # 输出张量列表    Список тензоров для выходов
    predicted_outputs = []  # 预测输出张量列表    Список тензоров для предсказанных выходов
    masks = []  # 掩码张量列表    Список тензоров для масок

    # 使用 values 属性快速访问数据    Использование свойства values для быстрого доступа к данным
    for i, row in enumerate(sentence_pairs.values):
        # 对输入（俄语）和输出（英语）进行分词    Токенизация входных данных (русский) и выходных данных (английский)
        input_tokens = tokenizer(row[1], max_length=max_len, padding='max_length', truncation=True, add_special_tokens=True)['input_ids']
        output_tokens = tokenizer(row[3], max_length=max_len - 1, padding='max_length', truncation=True, add_special_tokens=True)['input_ids']

        # 在输出序列前加上 BOS token    Добавление BOS-токена в начало выходной последовательности
        output_tokens_with_bos = [tokenizer.bos_token_id] + output_tokens
        # 在预测输出序列末尾加上 EOS token    Добавление EOS-токена в конец предсказанной выходной последовательности
        predicted_output_tokens = output_tokens + [tokenizer.eos_token_id]

        # 将 tokens 添加到列表中    Добавление токенов в списки
        inputs.append(torch.tensor(input_tokens))
        outputs.append(torch.tensor(output_tokens_with_bos))
        predicted_outputs.append(torch.tensor(predicted_output_tokens))  # 预测输出是没有 BOS token 的版本    Предсказанная выходная последовательность не содержит BOS-токена

        # 生成 mask，标记 pad 的位置为 0    Генерация маски, помечая места заполнения (pad) нулями
        masks.append(torch.tensor([1 if token != tokenizer.pad_token_id else 0 for token in output_tokens_with_bos]))

    # 使用 torch.stack 将列表转换为 tensor    Преобразование списков в тензоры с помощью torch.stack
    inputs = torch.stack(inputs)
    outputs = torch.stack(outputs)
    predicted_outputs = torch.stack(predicted_outputs)
    masks = torch.stack(masks)

    return (inputs, outputs), predicted_outputs, masks


# 使用自定义数据集函数生成数据    Генерация данных с помощью функции пользовательского датасета
(inputs, outputs), predicted_output, masks = Sentences_Dataset(data[['id_rus', 'text_rus', 'id_eng', 'text_eng']], tokenizer)
print("Inputs", inputs)
print("outputs", outputs)
print("Input tensor shape:", inputs.shape)       # 输入张量的形状    Форма входного тензора
print("Output tensor shape:", outputs.shape)     # 输出张量的形状    Форма выходного тензора
print("Mask tensor shape:", masks.shape)         # 掩码张量的形状    Форма тензора маски
print("Predicted Output tensor shape:", predicted_output.shape)  # 预测输出张量的形状    Форма тензора предсказанного выхода

Inputs tensor([[  140,   252, 43666,  ...,   141,   227, 15166],
        [  140,   242, 16142,  ..., 16843, 43108, 50258],
        [  140,   250, 22177,  ..., 50258, 50258, 50258],
        ...,
        [  140,   107, 12466,  ...,   111, 43666, 16142],
        [  140,   107, 12466,  ...,   111, 43666, 16142],
        [  140,   107, 12466,  ..., 50258, 50258, 50258]])
outputs tensor([[50257,  1890,  1752,  ..., 50258, 50258, 50258],
        [50257,    43,  1039,  ..., 50258, 50258, 50258],
        [50257,    40,   423,  ..., 50258, 50258, 50258],
        ...,
        [50257,    40,   760,  ..., 50258, 50258, 50258],
        [50257,    40,   760,  ..., 50258, 50258, 50258],
        [50257,    40, 17666,  ..., 50258, 50258, 50258]])
Input tensor shape: torch.Size([735543, 32])
Output tensor shape: torch.Size([735543, 32])
Mask tensor shape: torch.Size([735543, 32])
Predicted Output tensor shape: torch.Size([735543, 32])


In [8]:
# 定义自定义数据集类    Определение класса пользовательского датасета
class TranslationDataset(Dataset):
    def __init__(self, sentence_pairs, tokenizer, max_len=32):
        self.sentence_pairs = sentence_pairs  # 存储句子对数据    Хранение пар предложений
        self.tokenizer = tokenizer  # 存储分词器    Хранение токенизатора
        self.max_len = max_len  # 最大序列长度    Максимальная длина последовательности

        # 调用外部的 create_dataset 函数生成数据    Вызов внешней функции create_dataset для генерации данных
        (self.inputs, self.outputs), self.predicted_outputs, self.masks = Sentences_Dataset(sentence_pairs, tokenizer, max_len)

    def __len__(self):
        return len(self.inputs)  # 返回数据集的大小    Возвращение размера датасета

    def __getitem__(self, idx):
        return {  # 返回指定索引的数据    Возвращение данных по указанному индексу
            'input_ids': self.inputs[idx],  # 输入 ID    Входные ID
            'output_ids': self.outputs[idx],  # 输出 ID    Выходные ID
            'predicted_output_ids': self.predicted_outputs[idx],  # 预测输出 ID    Предсказанные выходные ID
            'attention_mask': self.masks[idx]  # 注意力掩码    Маска внимания
        }


# 创建数据集对象    Создание объекта датасета
translation_dataset = TranslationDataset(data[['id_rus', 'text_rus', 'id_eng', 'text_eng']], tokenizer)

# 创建数据加载器    Создание загрузчика данных
batch_size = 64  # 根据显存大小调整批量大小    Настройка размера батча в зависимости от доступной памяти
data_loader = DataLoader(translation_dataset, batch_size=batch_size, shuffle=True)  # 创建数据加载器对象    Создание объекта загрузчика данных

# 检查数据加载器    Проверка загрузчика данных
for batch in data_loader:
    print("Input IDs shape:", batch['input_ids'].shape)  # 打印输入 ID 的形状    Вывод формы входных ID
    print("Output IDs shape:", batch['output_ids'].shape)  # 打印输出 ID 的形状    Вывод формы выходных ID
    print("Predicted Output IDs shape:", batch['predicted_output_ids'].shape)  # 打印预测输出 ID 的形状    Вывод формы предсказанных выходных ID
    print("Attention mask shape:", batch['attention_mask'].shape)  # 打印注意力掩码的形状    Вывод формы маски внимания
    break  # 只查看第一个批次    Просмотр только первого батча

# 打印第一个样本的输入、输出、预测输出和注意力掩码    
# Вывод входных, выходных, предсказанных выходных данных и маски внимания для первого примера
for batch in data_loader:
    print("input_ids:", batch['input_ids'][0])  # 打印第一个样本的输入 IDs    Вывод входных ID первого примера
    print("output_ids:", batch['output_ids'][0])  # 打印第一个样本的输出 IDs    Вывод выходных ID первого примера
    print("Predicted Output IDs:", batch['predicted_output_ids'][0])  # 打印第一个样本的预测输出 IDs    Вывод предсказанных выходных ID первого примера
    print("attention_mask:", batch['attention_mask'][0])  # 打印第一个样本的注意力掩码    Вывод маски внимания первого примера
    break

Input IDs shape: torch.Size([64, 32])
Output IDs shape: torch.Size([64, 32])
Predicted Output IDs shape: torch.Size([64, 32])
Attention mask shape: torch.Size([64, 32])
input_ids: tensor([  140,   255, 20375, 15166,   220, 21727, 16142, 43108, 15166, 16843,
        12466,   111, 30143, 35072,   140,   109, 25443,   118, 15166, 16843,
        12466,   122,   140,   115, 16843, 21169, 15166, 12466,   110, 12466,
          107,   140])
output_ids: tensor([50257,  1212,   318,   262, 25420, 13546,   286,  2869, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258])
Predicted Output IDs: tensor([ 1212,   318,   262, 25420, 13546,   286,  2869, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258])
attention_mask:

In [9]:
def check_tokenizer_with_dataloader(data_loader, tokenizer, num_samples=5):
    # 获取一个批次数据    Получение одного пакета данных
    batch = next(iter(data_loader))
    
    # 提取 input_ids 和 output_ids    Извлечение input_ids и output_ids
    input_ids_batch = batch['input_ids']  # [batch_size, seq_len] 批次大小和序列长度    Размер пакета и длина последовательности
    output_ids_batch = batch['output_ids']  # [batch_size, seq_len] 批次大小和序列长度    Размер пакета и длина последовательности
    
    # 转为列表以便逐条处理    Преобразование в список для построчной обработки
    input_ids_list = input_ids_batch.tolist()
    output_ids_list = output_ids_batch.tolist()
    
    # 遍历样本并解码    Перебор примеров и декодирование
    for i, (input_ids, output_ids) in enumerate(zip(input_ids_list, output_ids_list)):
        # 使用 tokenizer 解码    Декодирование с использованием токенизатора
        decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)
        decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
        
        # 打印对比结果    Вывод результата сравнения
        print(f"Sample {i + 1}:")
        print(f"  Original Input IDs: {input_ids}")  # 原始输入 ID    Исходные входные идентификаторы
        print(f"  Decoded Input Text: {decoded_input}")  # 解码后的输入文本    Декодированный входной текст
        print(f"  Original Output IDs: {output_ids}")  # 原始输出 ID    Исходные выходные идентификаторы
        print(f"  Decoded Output Text: {decoded_output}")  # 解码后的输出文本    Декодированный выходной текст
        print()
        
        # 控制显示样本数量    Ограничение количества отображаемых примеров
        if i + 1 >= num_samples:
            break

In [10]:
# 检查分词器的工作情况    Проверка работы токенизатора
check_tokenizer_with_dataloader(data_loader, tokenizer, num_samples=5)  # 使用5个样本检查    Проверка на 5 примерах

Sample 1:
  Original Input IDs: [140, 94, 15166, 25443, 109, 141, 231, 18849, 12466, 95, 25443, 120, 35072, 12466, 122, 220, 20375, 25443, 120, 220, 141, 229, 20375, 15166, 12466, 123, 21169, 15166, 18849, 21727, 141, 227]
  Decoded Input Text: Сообщи Тому о том что происх
  Original Output IDs: [50257, 5756, 4186, 760, 45038, 5836, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258]
  Decoded Output Text: Let Tom know whats happening

Sample 2:
  Original Input IDs: [140, 241, 43666, 16843, 12466, 122, 22177, 12466, 121, 16142, 35072, 141, 229, 18849, 30143, 21727, 40623, 12466, 110, 25443, 112, 18849, 20375, 45367, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258]
  Decoded Input Text: Где он научился водить
  Original Output IDs: [50257, 8496, 750, 339, 2193, 284, 3708, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 502

# Model

In [11]:
# 编码器类，负责处理输入序列    Класс кодировщика, который обрабатывает входные последовательности
class Encoder(nn.Module):
    def __init__(self, input_vocab_size, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout):
        super().__init__()
        # 嵌入层，将输入词索引映射到向量    Слой эмбеддингов для отображения индексов слов в вектора
        self.embedding = nn.Embedding(input_vocab_size, embedding_dim)
        # 双向 GRU，用于提取上下文特征    Двунаправленный GRU для извлечения контекстных признаков
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional=True)
        # 将编码器的隐藏状态映射到解码器隐藏状态初值    Преобразование скрытых состояний кодировщика в начальное скрытое состояние декодера
        self.fc_hidden_init = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim)
        self.dropout = nn.Dropout(dropout)  # 添加 Dropout，防止过拟合    Добавление Dropout для предотвращения переобучения

    def forward(self, source_sequence):
        # 将序列转置为 (src_len, batch_size) 格式    Транспонирование последовательности в формат (src_len, batch_size)
        source_sequence = source_sequence.transpose(0, 1)
        # 嵌入并添加 Dropout，结果转置为 (src_len, batch_size, embedding_dim)    Эмбеддинг с добавлением Dropout, транспонируется в (src_len, batch_size, embedding_dim)
        embedded_sequence = self.dropout(self.embedding(source_sequence)).transpose(0, 1)
        # 输入嵌入到 GRU，得到输出和隐藏状态    Подача эмбеддингов в GRU, получение выходов и скрытых состояний
        encoder_outputs, encoder_hidden_states = self.rnn(embedded_sequence)
        # 合并双向隐藏状态，映射到解码器初始隐藏状态    Объединение двунаправленных скрытых состояний для начального состояния декодера
        decoder_initial_hidden = torch.tanh(
            self.fc_hidden_init(torch.cat((encoder_hidden_states[-2, :, :], encoder_hidden_states[-1, :, :]), dim=1))
        )
        return encoder_outputs, decoder_initial_hidden


# 注意力机制类，计算解码器隐藏状态与编码器输出的相关性    Класс механизма внимания, вычисляющий зависимость между скрытыми состояниями декодера и выходами кодировщика
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        # 解码器隐藏状态投影到编码器空间    Проекция скрытых состояний декодера в пространство кодировщика
        self.decoder_projection = nn.Linear(decoder_hidden_dim, encoder_hidden_dim * 2)
        self.scale = 1.0 / (encoder_hidden_dim * 2) ** 0.5  # 缩放因子，防止梯度消失    Коэффициент масштабирования для предотвращения затухания градиентов

    def forward(self, decoder_hidden, encoder_outputs):
        # 解码器隐藏状态：[batch_size, decoder_hidden_dim]    Скрытое состояние декодера: [batch_size, decoder_hidden_dim]
        # 编码器输出：[src_len, batch_size, encoder_hidden_dim * 2]    Выход кодировщика: [src_len, batch_size, encoder_hidden_dim * 2]
        src_len = encoder_outputs.shape[0]  # 获取源序列长度    Получение длины входной последовательности

        # 将解码器隐藏状态投影到编码器空间    Проекция скрытых состояний декодера в пространство кодировщика
        decoder_hidden = self.decoder_projection(decoder_hidden)
        decoder_hidden = decoder_hidden.unsqueeze(1)  # 添加时间维度：[batch_size, 1, encoder_hidden_dim * 2]
        encoder_outputs = encoder_outputs.transpose(0, 1)  # 转置编码器输出：[batch_size, src_len, encoder_hidden_dim * 2]
        # 计算注意力得分    Вычисление весов внимания
        attention_scores = torch.bmm(decoder_hidden, encoder_outputs.transpose(1, 2)) * self.scale
        # 对注意力得分进行 softmax 归一化    Нормализация весов внимания с помощью softmax
        attention_weights = F.softmax(attention_scores.squeeze(1), dim=1)
        return attention_weights


# 解码器类，负责生成目标序列    Класс декодера для генерации целевых последовательностей
class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout, attention):
        super().__init__()
        self.output_vocab_size = output_vocab_size  # 目标词汇表大小    Размер словаря целевых токенов
        self.attention = attention  # 注意力机制模块    Модуль механизма внимания
        self.embedding = nn.Embedding(output_vocab_size, embedding_dim)  # 嵌入层    Слой эмбеддингов
        # 解码器 GRU 输入包括上下文向量和目标嵌入    Вход GRU декодера включает контекстный вектор и эмбеддинги целевых токенов
        self.rnn = nn.GRU((encoder_hidden_dim * 2) + embedding_dim, decoder_hidden_dim)
        # 输出全连接层，用于生成目标词分布    Полносвязный слой для генерации распределения целевых токенов
        self.fc_out = nn.Linear((encoder_hidden_dim * 2) + decoder_hidden_dim + embedding_dim, output_vocab_size)
        self.dropout = nn.Dropout(dropout)  # Dropout 层    Слой Dropout

    def forward(self, target_token, decoder_hidden, encoder_outputs):
        target_token = target_token.unsqueeze(1)  # 添加时间维度：[batch_size, 1]    Добавление временного измерения: [batch_size, 1]
        # 嵌入目标 token，并应用 Dropout    Эмбеддинг целевых токенов с применением Dropout
        embedded_target = self.dropout(self.embedding(target_token)).transpose(0, 1)
        # 计算注意力权重    Вычисление весов внимания
        attention_weights = self.attention(decoder_hidden, encoder_outputs).unsqueeze(1)
        encoder_outputs = encoder_outputs.transpose(0, 1)  # 转置编码器输出    Транспонирование выходов кодировщика
        # 计算上下文向量    Вычисление контекстного вектора
        context_vector = torch.bmm(attention_weights, encoder_outputs).transpose(0, 1)
        # 将目标嵌入和上下文向量拼接作为 GRU 输入    Конкатенация эмбеддингов целевых токенов и контекстного вектора для входа в GRU
        rnn_input = torch.cat((embedded_target, context_vector), dim=2)
        rnn_output, hidden_state = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))  # 通过 GRU 计算    Вычисление с помощью GRU
        rnn_output = rnn_output.squeeze(0)  # 去掉时间维度    Удаление временного измерения
        embedded_target = embedded_target.squeeze(0)  # 去掉时间维度    Удаление временного измерения
        context_vector = context_vector.squeeze(0)  # 去掉时间维度    Удаление временного измерения
        # 计算输出预测    Вычисление предсказаний
        predictions = self.fc_out(torch.cat((rnn_output, context_vector, embedded_target), dim=1))
        return predictions, hidden_state.squeeze(0), attention_weights.squeeze(1)


# Seq2Seq 模型类，结合编码器、解码器和注意力机制    Класс Seq2Seq модели, объединяющий кодировщик, декодер и механизм внимания
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder  # 编码器    Кодировщик
        self.decoder = decoder  # 解码器    Декодер
        self.device = device

    def forward(self, source_sequence, target_sequence, teacher_forcing_ratio=0.9):
        batch_size = source_sequence.shape[1]  # 批量大小    Размер батча
        target_len = target_sequence.shape[0]  # 目标序列长度    Длина целевой последовательности
        target_vocab_size = self.decoder.output_vocab_size  # 目标词汇表大小    Размер словаря целевых токенов
        # 初始化输出张量    Инициализация тензора для выходов
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)
        # 通过编码器计算编码器输出和初始解码器隐藏状态    Получение выходов кодировщика и начального состояния декодера
        encoder_outputs, decoder_hidden = self.encoder(source_sequence)
        # 解码器的初始输入为 <BOS>（开始标记）    Начальный вход декодера - токен <BOS> (начало последовательности)
        decoder_input = target_sequence[0, :]
        for t in range(1, target_len):
            # 通过解码器预测下一个 token    Предсказание следующего токена с помощью декодера
            output, decoder_hidden, _ = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            outputs[t] = output  # 保存当前时间步的输出    Сохранение выхода текущего временного шага
            top1 = output.argmax(1)  # 获取概率最高的 token    Выбор токена с максимальной вероятностью
            # 决定是否使用教师强制    Решение о применении teacher forcing
            decoder_input = target_sequence[t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

In [12]:
# 定义输入和输出词汇表大小    Определение размеров словаря входной и выходной последовательностей
input_vocab_size = len(tokenizer.get_vocab())  # 输入词汇表大小    Размер словаря входных токенов
output_vocab_size = len(tokenizer.get_vocab())  # 输出词汇表大小    Размер словаря выходных токенов

# 定义编码器和解码器的嵌入维度    Определение размерности эмбеддингов кодировщика и декодера
encoder_embedding_dim = 256  # 编码器嵌入层的维度    Размерность эмбеддингов кодировщика
decoder_embedding_dim = 256  # 解码器嵌入层的维度    Размерность эмбеддингов декодера

# 定义编码器和解码器的隐藏状态维度    Определение размерности скрытых состояний кодировщика и декодера
encoder_hidden_dim = 256  # 编码器隐藏层的维度    Размерность скрытых состояний кодировщика
decoder_hidden_dim = 256  # 解码器隐藏层的维度    Размерность скрытых состояний декодера

# 定义 Dropout 概率，用于防止过拟合    Определение вероятности Dropout для предотвращения переобучения
encoder_dropout = 0.5  # 编码器的 Dropout 概率    Вероятность Dropout кодировщика
decoder_dropout = 0.5  # 解码器的 Dropout 概率    Вероятность Dropout декодера

# 实例化注意力机制模块    Инициализация модуля механизма внимания
attention = Attention(encoder_hidden_dim, decoder_hidden_dim)  # 创建注意力对象    Создание объекта механизма внимания

# 初始化编码器    Инициализация кодировщика
encoder = Encoder(
    input_vocab_size=input_vocab_size,  # 输入词汇表的大小    Размер словаря входных токенов
    embedding_dim=encoder_embedding_dim,  # 嵌入层的维度    Размерность эмбеддингов
    encoder_hidden_dim=encoder_hidden_dim,  # 隐藏状态的维度    Размерность скрытых состояний
    decoder_hidden_dim=decoder_hidden_dim,  # 解码器隐藏状态的维度，用于初始化解码器    Размерность скрытых состояний декодера для инициализации
    dropout=encoder_dropout  # Dropout 概率    Вероятность Dropout
)

# 初始化解码器    Инициализация декодера
decoder = Decoder(
    output_vocab_size=output_vocab_size,  # 输出词汇表的大小    Размер словаря выходных токенов
    embedding_dim=decoder_embedding_dim,  # 嵌入层的维度    Размерность эмбеддингов
    encoder_hidden_dim=encoder_hidden_dim,  # 编码器隐藏状态的维度，用于上下文向量    Размерность скрытых состояний кодировщика для контекстного вектора
    decoder_hidden_dim=decoder_hidden_dim,  # 解码器隐藏状态的维度    Размерность скрытых состояний декодера
    dropout=decoder_dropout,  # Dropout 概率    Вероятность Dropout
    attention=attention  # 注意力机制模块    Модуль механизма внимания
)

# 检查是否有 GPU 可用，并设置设备    Проверка наличия GPU и настройка устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 如果有 GPU 使用 GPU，否则使用 CPU    Использование GPU, если доступен, иначе CPU

# 初始化 Seq2Seq 模型    Инициализация Seq2Seq модели
seq2seq_model_attention = Seq2Seq(encoder, decoder, device).to(device)  # 将编码器和解码器传递给模型，并将其加载到设备上    Передача кодировщика и декодера в модель и перенос на устройство

# 打印模型结构    Печать структуры модели
print(seq2seq_model_attention)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(50260, 256)
    (rnn): GRU(256, 256, bidirectional=True)
    (fc_hidden_init): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (decoder_projection): Linear(in_features=256, out_features=512, bias=True)
    )
    (embedding): Embedding(50260, 256)
    (rnn): GRU(768, 256)
    (fc_out): Linear(in_features=1024, out_features=50260, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)


# Training

In [13]:
import gc  # 导入垃圾回收模块    Импорт модуля для управления сбором мусора

# 清除缓存和显存的函数    Функция для очистки кэша и видеопамяти
def clear_cache():
    gc.collect()  # 清理 Python 的垃圾内存    Очистка мусора в Python
    if torch.cuda.is_available():  # 如果 GPU 可用    Если доступен GPU
        torch.cuda.empty_cache()  # 清理 GPU 的缓存    Очистка кэша GPU

In [14]:
# 定义训练函数    Определение функции обучения
def train(seq2seq_model_attention, data_loader, optimizer, criterion, device, num_epochs, pad_token_id=50258):
    # 将模型设置为训练模式    Перевод модели в режим обучения
    seq2seq_model_attention.train()  
    for epoch in range(num_epochs):
        clear_cache()  # 清除缓存和显存    Очистка кэша и видеопамяти
        total_epoch_loss = 0  # 累积每个 epoch 的总损失    Суммарные потери за эпоху
        num_correct_tokens = 0  # 累积正确预测的 token 数    Суммарное количество правильно предсказанных токенов
        num_total_tokens = 0  # 累积总的 token 数    Суммарное количество токенов

        # 遍历数据加载器的每个批次    Итерация по батчам из загрузчика данных
        for batch in tqdm(data_loader, desc=f"Epoch [{epoch + 1}/{num_epochs}]"):
            # 获取输入和输出的 ID 并将它们传递到设备上    Извлечение ID входных и выходных последовательностей и их перенос на устройство
            input_ids = batch['input_ids'].to(device)  # 输入 ID    ID входных токенов
            output_ids = batch['output_ids'].to(device)  # 输出 ID    ID выходных токенов
            
            # 动态调整教师强制比例    Динамическое изменение коэффициента teacher forcing
            teacher_forcing_ratio = max(0.5, 1 - epoch * 0.1)  # 最低值为 0.5    Минимальное значение 0.5
            # 前向传播获取模型预测    Прямой проход для получения предсказаний модели
            predictions = seq2seq_model_attention(input_ids, output_ids, teacher_forcing_ratio=0.9)
            
            # 将目标序列调整为 1D    Приведение целевых последовательностей к одномерному виду
            target_ids = output_ids[:, 1:].contiguous().view(-1)  # 去掉 <BOS> 标记    Удаление токена <BOS>
            predictions = predictions[:, 1:].contiguous().view(-1, predictions.shape[-1])  # 调整预测的形状    Преобразование формы предсказаний
            
            # 计算当前批次的损失    Вычисление потерь для текущего батча
            loss = criterion(predictions, target_ids)
            
            optimizer.zero_grad()  # 清零优化器的梯度    Обнуление градиентов оптимизатора
   
            loss.backward()  # 反向传播计算梯度    Обратное распространение для расчета градиентов

            # 裁剪梯度以避免梯度爆炸    Ограничение градиентов для предотвращения их взрывного роста
            torch.nn.utils.clip_grad_norm_(seq2seq_model_attention.parameters(), max_norm=1.0)
    
            optimizer.step()  # 更新模型参数    Обновление параметров модели
  
            total_epoch_loss += loss.item()  # 累积损失    Суммирование потерь
            
            # 计算非 PAD token 的掩码    Создание маски для токенов, не являющихся PAD
            non_pad_mask = target_ids != 50258  # 忽略 PAD token    Игнорирование токенов PAD
            predicted_tokens = predictions.argmax(dim=1)  # 获取预测的 token    Получение предсказанных токенов
            # 计算正确预测的 token 数量    Подсчет правильно предсказанных токенов
            num_correct_tokens += (predicted_tokens == target_ids).masked_select(non_pad_mask).sum().item()
            # 累积非 PAD token 的总数量    Подсчет общего количества токенов, не являющихся PAD
            num_total_tokens += non_pad_mask.sum().item()

        # 计算当前 epoch 的平均损失和准确率    Вычисление средней потери и точности за текущую эпоху
        epoch_loss = total_epoch_loss / len(data_loader)  # 平均损失    Средняя потеря
        epoch_accuracy = num_correct_tokens / num_total_tokens  # 平均准确率    Средняя точность

        # 打印 epoch 的损失和准确率    Вывод потерь и точности за эпоху
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

In [15]:
# 定义损失函数，忽略 padding token 的损失计算
# Определяем функцию потерь, игнорируя padding токены
criterion = nn.CrossEntropyLoss(ignore_index=50258).to(device)  # 设置到指定设备 / Переносим на указанное устройство

# 定义优化器，使用 Adam 优化器，学习率为 0.01
# Определяем оптимизатор, используем Adam с learning rate = 0.002
optimizer = optim.Adam(seq2seq_model_attention.parameters(), lr=0.002)

# 训练模型  Обучение модели
train(seq2seq_model_attention, data_loader, optimizer, criterion, device, num_epochs=6)

Epoch [1/6]: 100%|█████████████████████████████████████████████████████████████| 11493/11493 [2:34:46<00:00,  1.24it/s]


Epoch [1/6] - Loss: 6.5875, Accuracy: 0.0634


Epoch [2/6]: 100%|█████████████████████████████████████████████████████████████| 11493/11493 [2:24:03<00:00,  1.33it/s]


Epoch [2/6] - Loss: 9.6336, Accuracy: 0.0271


Epoch [3/6]: 100%|█████████████████████████████████████████████████████████████| 11493/11493 [2:17:32<00:00,  1.39it/s]


...

Epoch [4/6] - Loss: 11.7274, Accuracy: 0.0178


Epoch [5/6]: 100%|█████████████████████████████████████████████████████████████| 11493/11493 [2:37:09<00:00,  1.22it/s]


Epoch [5/6] - Loss: 10.5979, Accuracy: 0.0166


Epoch [6/6]: 100%|█████████████████████████████████████████████████████████████| 11493/11493 [3:28:05<00:00,  1.09s/it]

Epoch [6/6] - Loss: 10.2495, Accuracy: 0.0167


# Testing

In [16]:
# 示例文本列表 - Пример списка текстов
sample_texts = [
    "Давайте что-нибудь попробуем!",  # 让我们试试吧！
    "Мне пора идти спать.",           # 我该睡觉了。
    "Что ты делаешь?",                # 你在做什么？
    "Что ты делаешь?",                # 你在做什么？
    "Собака бегает по траве",         # 狗在草地上奔跑
    "Небо и море - все синее."        # 天空和海洋全都是蓝色
]

In [17]:
# 定义翻译函数，输入文本，分词器，带注意力机制的 Seq2Seq 模型，最大翻译长度
# Определяем функцию перевода, принимая текст, токенизатор, Seq2Seq модель с вниманием и максимальную длину перевода
def translate(text, tokenizer, seq2seq_model_attention, max_len=24):
    seq2seq_model_attention.eval()  # 设置模型为评估模式  Устанавливаем модель в режим оценки
    seq2seq_model_attention.to(device)  # 将模型加载到指定设备  Переносим модель на заданное устройство

    # 将输入文本编码为张量并转移到设备
    # Кодируем входной текст в тензор и переносим на устройство
    source_sequence = tokenizer.encode(text, return_tensors='pt').squeeze(0).to(device)  # [source_len]
    source_sequence = source_sequence.unsqueeze(1)  # 为批处理添加维度  Добавляем размерность для батча: [source_len, 1]
    
    # 初始化编码器
    # Инициализируем энкодер
    encoder_outputs, decoder_hidden = seq2seq_model_attention.encoder(source_sequence)
    
    # 初始化解码器输入为 <BOS> 标记
    # Инициализируем вход декодера как токен <BOS>
    sos_token = tokenizer.bos_token_id  # 获取 <BOS> 标记的 ID  Получаем ID для токена <BOS>
    eos_token = tokenizer.eos_token_id  # 获取 <EOS> 标记的 ID  Получаем ID для токена <EOS>
    decoder_input = torch.tensor([sos_token], device=device)  # 解码器的输入为 <BOS> 标记  Вход декодера - токен <BOS>
    
    # 存储翻译结果
    # Храним результат перевода
    translated_tokens = []
    
    # 在最大长度内进行解码
    # Декодируем до максимальной длины
    for _ in range(max_len):
        # 解码一步
        # Шаг декодирования
        predictions, decoder_hidden, attention_weights = seq2seq_model_attention.decoder(decoder_input, decoder_hidden, encoder_outputs)

        predictions[0][translated_tokens] -= 1.0  # 降低已生成词的概率
        #print(f"Step {_}:")
        #print(f"Attention Weights: {attention_weights}")
        #print(f"predictions: {predictions}")
        # 获取概率最高的 token
        # Получаем токен с максимальной вероятностью
        top1 = predictions.argmax(1).item()  # 获取预测的最高概率的索引  Получаем индекс с наибольшей вероятностью
        
        # 保存生成的 token
        # Сохраняем сгенерированный токен
        translated_tokens.append(top1)
        
        # 如果遇到 <EOS> 标记，则停止生成
        # Прекращаем генерацию, если встретился токен <EOS>
        if top1 == eos_token:
            break
        
        # 下一步的解码器输入
        # Ввод для следующего шага декодирования
        decoder_input = torch.tensor([top1], device=device)  # 输入解码器为预测的 token  Ввод для декодера - это предсказанный токен
    
    # 使用 tokenizer.decode 将 token ID 转换为句子
    # Преобразуем ID токенов в предложение с помощью decode
    translated_sentence = tokenizer.decode(translated_tokens, skip_special_tokens=True)

    # 清理翻译结果
    # Очищаем результат перевода
    def clean_translation(translation):
        translation = re.sub(r'([!?.])\1+', r'\1', translation)  # 合并重复的标点符号  Объединяем повторяющиеся знаки препинания
        return translation.strip()  # 去除首尾空格  Убираем пробелы в начале и в конце

    return clean_translation(translated_sentence)  # 返回清理后的翻译结果  Возвращаем очищенный перевод

In [18]:
clear_cache()  # 清理缓存  Очистка кэша

# 使用模型对每个文本进行翻译
# Переводим каждый текст с помощью модели
for text in sample_texts:
    translation = translate(text, tokenizer, seq2seq_model_attention)
    print(f"Original: {text}")
    print(f"Translated: {translation}")
    print()

Original: Давайте что-нибудь попробуем!
Translated: started number color beautyes owe restaurant mother front whole owe restaurant owe restaurant card owe restaurant paint owe restaurant dead you saveFind

Original: Мне пора идти спать.
Translated: isUnfortunately enteringothsUnfortunately entering irresponsibleUnfortunately melThWillThWillTh frontHaveThWillThWillThWillThWill

Original: Что ты делаешь?
Translated: traveled suggest held suggest smoke suggest suggest speed suggest suggest sw suggest suggest suggest suggest suggest suggest suggest suggestriage suggest suggest shoeanni

Original: Что ты делаешь?
Translated: traveled suggest held suggest smoke suggest suggest speed suggest suggest sw suggest suggest suggest suggest suggest suggest suggest suggestriage suggest suggest shoeanni

Original: Собака бегает по траве
Translated: swurry conservative lookedter bet roughly exam suggest advise bet realityurry conservative looked conservative looked conservative looked conservative look